<a href="https://colab.research.google.com/github/zain711/dcipher/blob/main/vaccines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install bertopic
!pip install bertopic[visualization]
!pip install bert-extractive-summarizer
!pip install plotly --upgrade

  Using cached https://files.pythonhosted.org/packages/9d/2e/69579c3db25fa4f85d70a10f8a98d52c2b4a0dcbd153e8f17f425761bef4/plotly-4.14.2-py2.py3-none-any.whl
  Found existing installation: plotly 4.14.3
    Uninstalling plotly-4.14.3:
      Successfully uninstalled plotly-4.14.3
  Using cached https://files.pythonhosted.org/packages/1f/f6/bd3c17c8003b6641df1228e80e1acac97ed8402635e46c2571f8e1ef63af/plotly-4.14.3-py2.py3-none-any.whl
  Found existing installation: plotly 4.14.2
    Uninstalling plotly-4.14.2:
      Successfully uninstalled plotly-4.14.2


In [4]:
import json
import pandas as pd
from bertopic import BERTopic
from summarizer import Summarizer 
#from gensim.summarization import summarize
from tqdm import tqdm
import os
from sentence_transformers import SentenceTransformer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data = pd.read_json('/content/drive/MyDrive/vaccine-forums-tokenized.json')
#data.head(5)
docs = list(data.loc[:, "cleaned_post_text"].values)
times = list(data.loc[:, "post_published_at"].values)

In [6]:
clean_times = []
for a in times:
  ts = pd.to_datetime(str(a)) 
  d = ts.strftime('%Y.%m.%d')
  clean_times.append(d)

In [ ]:
len(docs)

26040

In [7]:
#topic_model = BERTopic()
#sentence_model = SentenceTransformer("stsb-roberta-large", device="cpu")
#topic_model = BERTopic(embedding_model="stsb-roberta-large")
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

In [8]:
topics, prob = topic_model.fit_transform(docs)

In [10]:
topic_model.get_topic_freq().head()

,Topic,Count
0,-1,12916
1,232,470
2,158,404
3,344,372
4,67,372


In [12]:
topic_model.get_topic(232)

[('wrong', 0.017142616132922943),
 ('cruel', 0.009917823083069873),
 ('unnecessary', 0.009184021131490607),
 ('disgusting', 0.007569614200653168),
 ('stupid', 0.007415703863097456),
 ('unsustainable', 0.006106586362159779),
 ('sorry', 0.006016776510963436),
 ('bitch', 0.005617245405599071),
 ('bad', 0.005381782205250917),
 ('inhumane', 0.005043493923750843)]

In [13]:
topic_model.visualize_topics()

In [14]:
topics_over_time = topic_model.topics_over_time(docs, topics, clean_times)

In [15]:
topic_model.visualize_topics_over_time(topics_over_time, top_n=5)